In [1]:
import random
from MuJoCo_Gym.mujoco_rl import MuJoCo_RL
from MuJoCo_Gym.single_agent_wrapper import Single_Agent_Wrapper
import time
from stable_baselines3 import PPO, SAC

In [2]:
def reward_function(mujoco_gym, agent):
    # Creates all the necessary fields to store the needed data within the dataStore at timestep 0 
    if "targets" not in mujoco_gym.dataStore[agent].keys():
        mujoco_gym.dataStore["targets"] = mujoco_gym.filterByTag("target")
        mujoco_gym.dataStore[agent]["current_target"] = mujoco_gym.dataStore["targets"][random.randint(0, len(mujoco_gym.dataStore["targets"]) - 1)]["name"]
        distance = mujoco_gym.distance(agent, mujoco_gym.dataStore[agent]["current_target"])
        mujoco_gym.dataStore[agent]["distance"] = distance
        new_reward = 0
    else: # Calculates the distance between the agent and the current target
        distance = mujoco_gym.distance(agent, mujoco_gym.dataStore[agent]["current_target"])
        new_reward = mujoco_gym.dataStore[agent]["distance"] - distance
        mujoco_gym.dataStore[agent]["distance"] = distance
    reward = new_reward * 10
    return reward

In [3]:
def done_function(mujoco_gym, agent):
    if mujoco_gym.dataStore[agent]["distance"] <= 1:
        return True
    else:
        return False

In [5]:
environment_path = "Environment/MultiEnvs.xml"
info_path = "Environment/info_example.json"
agents = ["agent1_torso"]
config_dict = {"xmlPath":environment_path, "infoJson":info_path, "agents":agents, "rewardFunctions":[reward_function], "doneFunctions":[done_function], "renderMode":False}
environment = MuJoCo_RL(config_dict)

/Users/cowolff/miniconda3/envs/Ray/lib/python3.9/site-packages/gymnasium/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [6]:
gymEnvironment = Single_Agent_Wrapper(environment, agents[0])

In [7]:
policy_kwargs = dict(net_arch=dict(pi=[4096, 4096, 2048], qf=[4096, 4096, 2048]))
model = SAC("MlpPolicy", gymEnvironment, verbose=1, train_freq=(1, "episode"), batch_size=256, learning_starts=10000, learning_rate=0.0015, buffer_size=1500000, policy_kwargs=policy_kwargs)
model.learn(total_timesteps=3000000, progress_bar=True)

Using mps device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/cowolff/miniconda3/envs/Ray/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

KeyboardInterrupt: 